author: Tim Liu

Dijkstra`s algorithm can be seen as derivation of the breadth-first search
It introduces the idea of **cost**.  
We mentioned that BFS search **uniformly**, Dijkstra includes the factor of **distance to the start**

In [ ]:
from queue import PriorityQueue
import time
from math import inf, sqrt, abs
from utils import env, plotting

class Djikstra():
    def __init__(self) -> None:
        self.frontier = PriorityQueue()
        self.parent = dict()
        self.g = dict() # a critical change compared with BFS
        self.env = env.Env()
        self.path = []

        self.start = (1,1)
        self.goal = (self.env.x_size - 2, self.env.y_size - 2)

        self.frontier.put((0, self.start))
        self.g[self.start] = 0
        self.parent[self.start] = None

The class initialization is almost the same **except for** we introduce a dictionary named g, and frontier becomes a **priority queue**.  
In python, the priority queue prioritize the small value in the queue. The underlying machanism is heap. In C++, there is std::priority_queue  
- An implementation detail: we must put the g value before the node`s pair, so the priority queue can sort with the g value.

In [ ]:
class Djikstra(Djikstra):
    def cal_dist(self, nA, nB):
        if nA in self.env.obs or nB in self.env.obs:
            return inf
        if len(self.env.motions) == 8:
            return sqrt((nA[0] - nB[0]) ** 2 + (nA[1] - nB[1]) ** 2)
        else:
            return abs(nA[0] - nB[0]) + abs(nA[1] - nB[1])

So, how to calculate distance? In Dijkstra, this depends on available directions. Euclidian distance for 8 or Manhattan distance for 4.

In [ ]:
class Djikstra(Djikstra):
    def search(self):
        found = False
        while not self.frontier.empty():
            cur_node = self.frontier.get()[1]
            if cur_node == self.goal:
                found = True
                break
            for new_node in self.get_neighbor(cur_node):

                new_cost = self.g[cur_node] + self.cal_dist(cur_node, new_node)

                if new_node not in self.g or new_cost < self.g[new_node]:
                    self.g[new_node] = new_cost
                    self.parent[new_node] = cur_node                
                    self.frontier.put((self.g[new_node], new_node))
        if found:
            self.get_path()
            return self.path, self.parent
        else:
            return None, None

This is the searching process. The thoughts of **traversing the queue`s head** and function structure are the same with BFS.  
However, for neighbouring each node, we **always** calculate its cost (distance to the start). This inevitably causes some repeated calculation, going over the same node for multiple times.  
We want to prioritize the nearest node (the essence of Dijkstra), so always **update** the distance when the new cost is smaller. smaller new cost may be caused by obstacles.  
But, when to update the __*parent*__ and __*frontier*__? This was once confusing for me. 
- For *parent*, if and only if the *new_cost* from *new_node* is smaller, we then update. Or else this node can just remain its ancestor.  
- For *frontier*, if and only if the *new_cost* from *new_node* is smaller, we **retake** it into *frontier*, to make it be considered in the next turn. Or else the node has been traversed, thus no need to calculate again.


In [ ]:
class Djikstra(Djikstra):
    def get_neighbor(self, s):
        l = list()
        for move in self.env.motions:
            if (s[0] + move[0], s[1] + move[1]) not in self.env.obs:
                l.append((s[0] + move[0], s[1] + move[1]))
        return l
class Djikstra(Djikstra):
    def get_path(self):
        cur = self.goal
        while cur != self.start:
            self.path.append(cur)
            cur = self.parent[cur] # iterate until reaching the start
        self.path.reverse() # note that the constructed path is from goal to start, in reverse order
        print("path found!")
def main():
    planner = Djikstra()
    t_start = time.time()
    path, visited = planner.search();
    t_end = time.time();
    print(t_end - t_start)

if __name__=="__main__":
    main()

Congrats! The rest is the same with the BFS. And we (at least me XD) can call it a day!